In [13]:
import datetime
import json
import random
from pathlib import Path

import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

ModuleNotFoundError: No module named 'config.LLMParticipantConfig'; 'config' is not a package

In [3]:







    def save_results(model, description, format_version):

        description = input("Enter a description for this trial: ")

        header = {
        "time": datetime.datetime.isoformat(datetime.datetime.now()),
        "description": description,
        "model": MODEL,
        "example": EXAMPLE,
        "format_version": FORMAT_VERSION,
        }

In [16]:
class LLMParticipant:

    def __init__(
            self,
            config: dict
        ) -> None:

        self.config = config
        self.MODEL = config.MODEL
        self.TEMPERATURE = config.TEMPERATURE
        self.EXAMPLE = config.EXAMPLE
        self.FORMAT_VERSION = config.FORMAT_VERSION
        self.SYSTEM_CONTENT = config.SYSTEM_CONTENT
        self.INPUTS = config.INPUTS

        self.data = self.load_input_questions()


        # self.tokenizer = AutoTokenizer.from_pretrained(MODEL)
        # self.model = AutoModelForCausalLM.from_pretrained(MODEL,
        #     # token='hf_xxx'
        #     device_map="auto",
        #     trust_remote_code=True,
        #     torch_dtype=torch.bfloat16,
        #     local_files_only=True
        # )

        # self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        # self.model.to(device)

        # self.tokenizer.pad_token = tokenizer.eos_token
        # self.tokenizer.pad_token_id = tokenizer.eos_token_id
        # self.tokenizer.padding_side = 'left'

    def load_input_questions(
            self,
        ) -> dict:

        with self.INPUTS.open(encoding='utf-8') as f:
            data = json.load(f)

        return data

    def assemble_messages(
            self,
            data: dict,
        ) -> list[list[dict]]:

        inputs = []
        for example in data.values():

            user_content = ""
            user_content += f"Question: {example['question']}\n\nProcess: "
            for step in example['explanation']:
                user_content += f"{step['step']}. {step['explanation']}. "

            messages = [
                {
                    "role": "system",
                    "content": self.SYSTEM_CONTENT
                },
                {
                    "role": "user",
                    "content": user_content
                }
            ]

            inputs.append(messages)

        return inputs

    def apply_template_generate_response(
            self,
            inputs
        ) -> list[str]:

        input_messages = self.tokenizer.apply_chat_template(inputs, padding=True, tokenize=True, return_tensors="pt", add_generation_prompt=True).to(device)
        outputs = self.model.generate(input_messages, max_new_tokens=256, temperature=self.TEMPERATURE)
        responses = [self.tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

        return responses

    def save_results(
            self,
            results: list[dict]
        ):

        outputs = {
            "config": self.config,
            "results": results
        }

        output_filename = datetime.datetime.isoformat(datetime.datetime.now())

        with Path("outputs", f"{self.INPUTS.stem}", f"{output_filename}.json").open('w', encoding='utf-8') as f:
            json.dump(outputs, f, indent=4)

    def participate(
            self
        ):

        inputs = self.assemble_messages(self.data)
        responses = self.apply_template_generate_response(inputs)

        outputs = []
        for q, r in zip(self.data.keys(), responses):
            t = self.data[q]['template_id']
            outputs.append({
                "question_id": q,
                "template_id": t,
                "response": r,
            })

        self.save_results(outputs)

        return outputs


In [15]:
MODEL = 'nfs/public/hf/models/meta-llama/Meta-Llama-3-8B-Instruct'
TEMPERATURE = 0.3
EXAMPLE = '[1, 2, 3, 4]'
FORMAT_VERSION = 2
SYSTEM_CONTENT = f"Your role is to select, from a list the steps, those that are most important for inclusion in a summary explanation of that process. Format your output as a list, for example {EXAMPLE}. Output only this short summary paragraph and nothing else."
INPUTS = 'select.json'
DESCRIPTION = input("Enter a description for this trial: ")

config = {
    "MODEL": MODEL,
    "TEMPERATURE": TEMPERATURE,
    "EXAMPLE": EXAMPLE,
    "FORMAT_VERSION": FORMAT_VERSION,
    "SYSTEM_CONTENT": SYSTEM_CONTENT,
    "INPUTS": INPUTS,
    "DESCRIPTION": DESCRIPTION
}

In [17]:
P = LLMParticipant(config=config)